In [140]:
#Import the basic standard template libraries

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [141]:
#Identify the files that will be used for the ML alrgorythm

path = "C:\\Users\\Todd\\.kaggle\\competitions\\house-prices-advanced-regression-techniques"
files = os.listdir(path)

In [142]:
#Shows the files in the folder

files

['data_description.txt', 'sample_submission.csv', 'test.csv', 'train.csv']

In [143]:
#Generate the full file path

files_path = [path + "\\" +  file for file in files]

In [144]:
#Read the housing CSV files

read = lambda x: pd.read_csv(x)
test_housing, train_housing= read(files_path[2]),read(files_path[3])
cat_text = open(path + "\\" + files[0])

In [145]:
cat_text.read();

In [146]:
train_housing.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [147]:
#Look at the correlation from the data

len(train_housing.columns)
train_housing.corr()["SalePrice"].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

In [148]:
#Function used to seperate the data into indipendant and dependant vectors

def seperate_data(housingData):
    
    X = housingData.drop("SalePrice", axis=1)
    
    y = housingData["SalePrice"]
    
    return X, y

In [149]:
#This cell is not necesarry since "cross_val_score" is used in a cell below and is far more efficient

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(train_housing,
 test_size=0.2, random_state=42)

In [150]:
housing, y = seperate_data(train_housing)
housing.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [151]:
#This function seperates the data into categorical data and numerical data

def object_cat(df):
    object_cat = []
    numeric_cat = []
    for cat in df:
        
        # Using df["MSZoning"].dtypes is a workaround for identifying the Pandas object datatype
        
        if df[cat].dtypes == df["MSZoning"].dtypes:
            
            #If data in the calumn is full of object (i.e. categorical) then it is placed in object_cat
            object_cat.append(cat)
        else:
            
            #Else the column data is placed into numerical_cat since only objects and numerical data exist in
            # the data set
            numeric_cat.append(cat)
            
    #Return a tuple of lists
    return object_cat, numeric_cat

#Used function
cat_labels, num_labels = object_cat(housing)



In [152]:
housing.columns == test_housing.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [133]:
# from sklearn.preprocessing import OrdinalEncoder
# enc = OrdinalEncoder()
# encoded_housing_cat = enc.fit_transform(cat_housing_filled)

# housing_prepared = housing.copy()
# housing_prepared.loc[:, cat_labels] = encoded_housing_cat

In [134]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder()),
])

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])


full_pipeline = ColumnTransformer([
    ("num_pipeline", num_pipeline, num_labels),
    ("cat_pipeline", cat_pipeline, cat_labels)
])




housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

#num_housing = housing[num_labels]
#num_imp = SimpleImputer(strategy='median')
#housing_prepared.loc[:,num_labels] = num_imp.fit_transform(num_housing)

#num_pipeline.fit_transform(num_housing)

housing_prepared.shape

(1460, 80)

In [135]:
from sklearn.ensemble import RandomForestRegressor

X = housing_prepared

rfr_housing = RandomForestRegressor(n_estimators=429,max_features=16)
rfr_housing.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=16, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=429, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [136]:
from sklearn.linear_model import LinearRegression

lr_housing = LinearRegression()
lr_housing.fit(X,y)





LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [96]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rfr_housing, X, y, scoring="neg_mean_squared_error", cv = 10)

In [98]:
rfr_scores = np.sqrt(-scores)
rfr_scores

scores = cross_val_score(lr_housing, X, y, scoring="neg_mean_squared_error", cv = 10)
lr_scores = np.sqrt(-scores)

In [92]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=250, high=450),
        'max_features': randint(low=10, high=20),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(X, y)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001A3BD9DE128>, 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001A3BD9FABA8>},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error',
          verbose=0)

In [94]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

29030.333975945785 {'max_features': 16, 'n_estimators': 429}
28926.769627562346 {'max_features': 17, 'n_estimators': 438}
28936.52954977957 {'max_features': 14, 'n_estimators': 352}
28974.422409295632 {'max_features': 19, 'n_estimators': 324}
28958.659867566235 {'max_features': 17, 'n_estimators': 366}
29053.65400157596 {'max_features': 13, 'n_estimators': 353}
28913.321387640623 {'max_features': 17, 'n_estimators': 380}
29031.34949531161 {'max_features': 15, 'n_estimators': 302}
29176.989291146216 {'max_features': 11, 'n_estimators': 337}
29032.885544931676 {'max_features': 15, 'n_estimators': 379}


In [137]:
def show_my_scores(scores):
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())
    print("Scores:", scores)

In [104]:
print("Scores for Random Forest Regressor")
show_scores(rfr_scores)
print("\nScores for Linear Regressor")
show_scores(lr_scores)

Scores for Random Forest Regressor
Scores: [21255.95574729 25878.11958319 22780.88237208 39440.47899391
 33914.65373933 25629.60361244 22683.11795434 22858.24020461
 38627.44712693 24083.84084637]
Mean: 27715.234018047086
Standard Deviation: 6564.183974330773

Scores for Linear Regressor
Scores: [2.26828259e+04 3.48133739e+04 2.61398424e+04 4.24689635e+04
 3.60829038e+04 2.91142481e+04 2.08841193e+13 2.69418338e+04
 6.65765231e+04 3.42422940e+04]
Mean: 2088411963282.7168
Standard Deviation: 6265235783493.883


In [108]:
def test_model(data):
    housing, y = seperate_data(data)
    cat_labels, num_labels = object_cat(housing)
    housing_prepared = full_pipeline.fit_transform(housing)
    X = housing_prepared
    rfr_housing = RandomForestRegressor(n_estimators=429,max_features=16)
    rfr_housing.fit(X,y)
    scores = cross_val_score(rfr_housing, X, y, scoring="neg_mean_squared_error", cv = 10)
    rfr_scores = np.sqrt(-scores)
    print("Scores for Random Forest Regressor")
    show_my_scores(rfr_scores)
    

In [109]:
test_model(train_housing)

Scores for Random Forest Regressor
Mean: 27953.234452540502
Standard Deviation: 6677.533024368918
Scores: [21291.14067847 25614.04572981 22595.25472069 39270.98495886
 34867.46278479 25956.09902498 23031.0814486  22878.8043659
 39241.87078199 24785.60003131]


In [112]:
def predict_values(data):
    cat_labels, num_labels = object_cat(data)
    housing_prepared = full_pipeline.fit_transform(housing)
    X = housing_prepared
    rfr_housing = RandomForestRegressor(n_estimators=429,max_features=16)
    rfr_housing.fit(X,y)
    #scores = cross_val_score(rfr_housing, X, y, scoring="neg_mean_squared_error", cv = 10)
    #rfr_scores = np.sqrt(-scores)
    print("Scores for Random Forest Regressor")
    print("")
    

In [113]:
predict_values(test_housing)

Scores for Random Forest Regressor
Mean: 27811.2552789634
Standard Deviation: 6589.6262283061415
Scores: [21307.68081472 25654.64437762 22579.89520982 39949.5096041
 34180.46274727 25520.26011813 22896.2016915  23313.37831405
 38328.63398877 24381.88592364]


In [161]:

test_prepared = full_pipeline.fit_transform(test_housing)
predicted_value = rfr_housing.predict(test_prepared)
predicted_value

array([128353.87878788, 155792.6037296 , 184582.40792541, ...,
       159801.69230769, 118073.01864802, 239765.5011655 ])

In [164]:
test_housing_predicted_price = test_housing.copy()
test_housing_predicted_price["Predicted Price"] = predicted_value.round(2)

In [167]:
test_housing_predicted_price

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Predicted Price
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,128353.88
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,155792.60
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,184582.41
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,187309.68
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,195318.14
5,1466,60,RL,75.0,10000,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,188798.43
6,1467,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,GdPrv,Shed,500,3,2010,WD,Normal,173932.37
7,1468,60,RL,63.0,8402,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,175876.99
8,1469,20,RL,85.0,10176,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2010,WD,Normal,179416.71
9,1470,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,4,2010,WD,Normal,127456.48
